In [1]:
from __future__ import print_function
from fenics import *
import numpy as np
import matplotlib.pyplot as plt
from dolfin import *
from mshr import *
from ufl import as_tensor
from ufl import Index
import math
import mshr

parameters["allow_extrapolation"] = True
parameters["form_compiler"]["cpp_optimize"] = True
set_log_level(20)

# SOLVE THE HEAT TRANSFER PROBLEM FOR A 3D TARGET


#----------------
t_start = 0.0
t_end = 200
nstep = 200
dtime = (t_end - t_start)/ nstep  #time step
#-----------------
#Configure for dimensions of system
#----------THERMAL--PROPERTIES--[Element: W]--------
kappa = 0.0172         #conductivity [W/mm K] 
c = 0.132         #Specific Heat Capacity [J/gK]
rho = 0.01925         #Density [g/mm^3]
const = kappa /(c * rho)
tau_T = 0.0 #Temperature Gradient Lag
tau_q = 0.0 #Heat Flux Lag
#---------------------------------------


pi = 3.141592653589793
T_am = 298 #ambient vacuum temperature (K)
T_a4 = T_am**4
epsilon = 0.02  # material emissivity
sigma = 5.67E-14 # W/(mm**2.K**4)
es = epsilon*sigma
Pmax = 400 #peak power of laser in W
w = 0.1     #mm
R = 1.5     #mm
area = pi*R*R  #area of target (mm^2)
depth = 8.0 #thickness of target (mm)
volume = area * depth #define a disk through which the laser is absorbed
Iden = Pmax / (pi*w*w*volume*rho)    #Intensity per unit mass
absorb_depth = 1e-7

Laser = Expression('2*A*Iden*exp(-pow((x[0] - 0), 2)/(w*w)-pow((x[1]-0), 2)/(w*w)-pow((x[2]-0), 2)/(w2*w2))',degree=3, A=1, Iden=Iden,w= 0.5, w2= absorb_depth) #power (w2 localises the z-coordinates)
#A =! emissivity for system in thermal equillibrium

geometry = mshr.Cylinder(Point(0,0,0),Point(0,0,-8),R,R) #8mm long target 
 
# Create mesh
mesh = generate_mesh(geometry, 40)          # generate a mesh from the given geometry
Space = FunctionSpace((mesh), 'P', 1)      #define finite element function space, defined via basis functions
VectorSpace = VectorFunctionSpace(mesh, 'P', 1)
cells = MeshFunction('size_t',mesh,mesh.topology().dim()) #codimension of 0
facets = MeshFunction('size_t',mesh,mesh.topology().dim()-1) #codimension of 1
da = Measure('ds', domain=mesh, subdomain_data = facets)  #area element
dv = Measure('dx', domain=mesh, subdomain_data = cells)   #volume element

initial_T = Expression("Tini", Tini=T_am, degree=3) # extrapolate an expression for the temperature before heating
T0 = interpolate(initial_T, Space)
T = Function(Space)         # Temperature
V = TestFunction(Space)     # Test Function used for FEA
dT = TrialFunction(Space)   # Temperature Derivative
q0 = Function(VectorSpace)  # heat flux at previous time step
i = Index()
G = as_tensor(T.dx(i), (i))  #gradient of T
G0 = as_tensor(T0.dx(i), (i)) # gradient of T at previous time step 

q = as_tensor(dtime/(dtime + tau_q) * (tau_q/dtime*q0[i] - kappa*(1+tau_T/dtime)*G[i] + kappa*tau_T/dtime*G0[i]),(i)) #heat
F = (rho*c/dtime*(T-T0)*V - q[i]*V.dx(i) - rho*Laser*V ) * dv + es*(T**4 - T_a4)*V*da   #final form to solve
Gain = derivative(F, T, dT)    # Gain will be usedas the Jacobian required to determine the evolution of a dynamic system 

file_T = File('target3D/solution.pvd')
for t in np.arange(t_start,t_end,dtime):
	print( "Time", t)
	solve(F==0, T, [], J = Gain, solver_parameters={"newton_solver":{"linear_solver": "mumps", "relative_tolerance": 1e-3} }, form_compiler_parameters={"cpp_optimize": True, "representation": "quadrature","quadrature_degree": 2} )
	file_T << (T,t)
	q_tmp = project(q, VectorSpace)
	q0.assign(q_tmp)
	T0.assign(T)      #change so that T0 is equal to T for the next time step

Calling FFC just-in-time (JIT) compiler, this may take some time.
Generating mesh with CGAL 3D mesh generator
Calling FFC just-in-time (JIT) compiler, this may take some time.
Time 0.0
Calling FFC just-in-time (JIT) compiler, this may take some time.


/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/quadrature/quadraturerepresentation.py:58: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()


  Ignoring precision in integral metadata compiled using quadrature representation. Not implemented.
  Ignoring precision in integral metadata compiled using quadrature representation. Not implemented.
Calling FFC just-in-time (JIT) compiler, this may take some time.


/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/quadrature/quadraturerepresentation.py:58: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
  Ignoring precision in integral metadata compiled using quadrature representation. Not implemented.
  Ignoring precision in integral metadata compiled using quadrature representation. Not implemented.


...

/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()
/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()
/opt/miniconda3/envs/fenicsproject/lib/python3.12/site

  Newton iteration 0: r (abs) = 2.151e-07 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-03)
  Newton iteration 1: r (abs) = 2.615e-13 (tol = 1.000e-10) r (rel) = 1.216e-06 (tol = 1.000e-03)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Time 196.0
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.975e-07 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-03)
  Newton iteration 1: r (abs) = 2.552e-13 (tol = 1.000e-10) r (rel) = 1.292e-06 (tol = 1.000e-03)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Time 197.0
Solving nonlinear variational problem.


/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()
/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()


  Newton iteration 0: r (abs) = 1.814e-07 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-03)
  Newton iteration 1: r (abs) = 2.557e-13 (tol = 1.000e-10) r (rel) = 1.409e-06 (tol = 1.000e-03)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Time 198.0
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.666e-07 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-03)
  Newton iteration 1: r (abs) = 2.548e-13 (tol = 1.000e-10) r (rel) = 1.529e-06 (tol = 1.000e-03)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Time 199.0
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.530e-07 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-03)
  Newton iteration 1: r (abs) = 2.496e-13 (tol = 1.000e-10) r (rel) = 1.631e-06 (tol = 1.000e-03)
  Newton solver finished in 1 iterations and 1 linear solver iterations.


/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()
/opt/miniconda3/envs/fenicsproject/lib/python3.12/site-packages/ffc/jitcompiler.py:234: QuadratureRepresentationDeprecationWarning: 
*** ===================================================== ***
*** FFC: quadrature representation is deprecated! It will ***
*** likely be removed in 2018.2.0 release. Use uflacs     ***
*** representation instead.                               ***
*** ===================================================== ***
  issue_deprecation_warning()
/opt/miniconda3/envs/fenicsproject/lib/python3.12/site